# <center>World Map Geovisualization using Folium and Covid Data
  ## <center> Author : Shivani Rana
   

### Import Libraries

In [3]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

### Access Data via API

In [10]:
conn=http.client.HTTPSConnection("api.covid19api.com")
payload=''
headers={}
conn.request("GET","/summary",payload,headers)
res=conn.getresponse()
data=res.read().decode("UTF-8")

### Convert Data to JSON

In [9]:
covid1=json.loads(data)

### Normalize Data 

In [14]:
pd.json_normalize(covid1["Countries"],sep=",")

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,210,45490,13,1725,23,36145,2020-11-26T16:27:02Z
1,Albania,AL,albania,644,34944,8,743,365,17031,2020-11-26T16:27:02Z
2,Algeria,DZ,algeria,1025,78025,20,2329,642,50712,2020-11-26T16:27:02Z
3,Andorra,AD,andorra,77,6428,0,76,39,5542,2020-11-26T16:27:02Z
4,Angola,AO,angola,79,14821,2,340,73,7517,2020-11-26T16:27:02Z
...,...,...,...,...,...,...,...,...,...,...
186,Viet Nam,VN,vietnam,5,1321,0,35,0,1153,2020-11-26T16:27:02Z
187,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-11-26T16:27:02Z
188,Yemen,YE,yemen,10,2124,2,611,7,1474,2020-11-26T16:27:02Z
189,Zambia,ZM,zambia,69,17535,0,357,48,16755,2020-11-26T16:27:02Z


### Convert to Pandas DataFrame

In [15]:
df=pd.DataFrame(covid1['Countries'])
df

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,210,45490,13,1725,23,36145,2020-11-26T16:27:02Z,{}
1,Albania,AL,albania,644,34944,8,743,365,17031,2020-11-26T16:27:02Z,{}
2,Algeria,DZ,algeria,1025,78025,20,2329,642,50712,2020-11-26T16:27:02Z,{}
3,Andorra,AD,andorra,77,6428,0,76,39,5542,2020-11-26T16:27:02Z,{}
4,Angola,AO,angola,79,14821,2,340,73,7517,2020-11-26T16:27:02Z,{}
...,...,...,...,...,...,...,...,...,...,...,...
186,Viet Nam,VN,vietnam,5,1321,0,35,0,1153,2020-11-26T16:27:02Z,{}
187,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-11-26T16:27:02Z,{}
188,Yemen,YE,yemen,10,2124,2,611,7,1474,2020-11-26T16:27:02Z,{}
189,Zambia,ZM,zambia,69,17535,0,357,48,16755,2020-11-26T16:27:02Z,{}


### Drop Unnecessary Columns

In [16]:
covid2=df.drop(columns=["CountryCode","Slug","Premium","Date"],axis=1)
covid2

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,210,45490,13,1725,23,36145
1,Albania,644,34944,8,743,365,17031
2,Algeria,1025,78025,20,2329,642,50712
3,Andorra,77,6428,0,76,39,5542
4,Angola,79,14821,2,340,73,7517
...,...,...,...,...,...,...,...
186,Viet Nam,5,1321,0,35,0,1153
187,Western Sahara,0,10,0,1,0,8
188,Yemen,10,2124,2,611,7,1474
189,Zambia,69,17535,0,357,48,16755


### Generate Base Map

In [18]:
m=folium.Map(tiles="Stamen Terrain",min_zoom=1.5)
m

### Obtain Geodata

geodata url: https://raw.githubusercontent.com/python-visualization/folium/master/examples/data

geo data file: world-countries.json

In [19]:
url="https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
country_shapes=f'{url}/world-countries.json'


### Generate Choropleth Map Layer

In [22]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name="Covid-19",
    data=covid2,
    columns=["Country","TotalConfirmed"],
    key_on='feature.properties.name',
    fill_color="OrRd",
    nan_fill_color="black",
    legend_name="Total Confirmed covid cases"
).add_to(m)
m

### Generate Circular Markers

In [23]:
covid2.update(covid2['TotalConfirmed'].map("Total Confirmed:{}".format))
covid2.update(covid2['TotalRecovered'].map("Total Recovered:{}".format))

coordinates of countries: https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv

In [26]:
cord=pd.read_csv("https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv")
cord

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra
...,...,...,...
239,-13.768752,-177.156097,Wallis and Futuna
240,24.215527,-12.885834,Western Sahara
241,15.552727,48.516388,Yemen
242,-13.133897,27.849332,Zambia


In [28]:
covid_final=pd.merge(covid2,cord,on='Country')

In [29]:
def PlotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                       radius=5,
                       weight=2,
                       popup=[point.Country,point.TotalConfirmed,point.TotalRecovered],
                       fill_color="#000000").add_to(m)

In [33]:
covid_final.apply(PlotDot,axis=1)
m.fit_bounds(m.get_bounds())
m

### Generate Base Map

In [34]:
m1=folium.Map(tiles='StamenToner',min_zoom=2)
m1

In [36]:
deaths=covid_final['TotalDeaths'].astype(float)

In [39]:
lat=covid_final['latitude'].astype(float)

In [41]:
lon=covid_final['longitude'].astype(float)

### Generate Heat Map Layer

In [46]:
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0))

### Generate Circular Marker

In [47]:
def PlotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                       radius=5,
                       weight=2,
                       popup=[point.Country,point.TotalDeaths],
                       fill_color="#000000").add_to(m1)
covid_final.apply(PlotDot,axis=1)
m1.fit_bounds(m1.get_bounds())
m1